In [1]:
#Notebook description

# this notebook is intended to track and price commodities based on the demand and supply data

In [2]:
#Load libraries 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True
logger.propagate = False
# Load libraries
from Quantapp.Plotter import Plotter
from Quantapp.Computation import Computation
from Quantapp.EconomicData import EconomicData

import numpy as np
import json
import os
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
from plotly.subplots import make_subplots
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#shut down warnings
import warnings
warnings.filterwarnings("ignore")


qc = Computation()
qp = Plotter()
qe = EconomicData()

In [7]:
#Load parameters
market_benchmark = yf.Ticker("SPY").history(period="5y")
commodities_benchmark_broad = yf.Ticker("DBC").history(period="5y")
commodities_benchmark_energy = yf.Ticker("DBE").history(period="5y")
commodities_benchmark_metals = yf.Ticker("DBB").history(period="5y")
commodities_benchmark_precious_metals = yf.Ticker("DBP").history(period="5y")
commodities_benchmark_agriculture = yf.Ticker("DBA").history(period="5y")

commodities_dict_energy = {
    "Crude Oil": "CL=F",
    "Natural Gas": "NG=F",
    "Heating Oil": "HO=F",
    "Gasoline": "RB=F"
}



commodities_dict_metals = {
    "Gold": "GC=F",
    "Silver": "SI=F",
    "Copper": "HG=F",
    "Platinum": "PL=F",
    "Palladium": "PA=F"
}

commodities_dict_precious_metals = {
    "Gold": "GC=F",
    "Silver": "SI=F",
    "Platinum": "PL=F",
    "Palladium": "PA=F"
}

commodities_dict_agriculture = {
    "Wheat": "ZW=F",
    "Corn": "ZC=F",
    "Soybeans": "ZS=F",
    "Soybean Oil": "ZL=F",
    "Soybean Meal": "ZM=F",
    "Oats": "ZO=F",
    "Rough Rice": "ZR=F",
    "Cotton": "CT=F",
    "Oranges": "OJ=F",
    "Sugar": "SB=F",
    "Live Cattle": "LE=F",
    "Lean Hogs": "HE=F",
    "Feeder Cattle": "GF=F"
}

#turn commodities_dict_energy into a dataframe with closing prices
commodities_df_energy = pd.DataFrame()

for key, value in commodities_dict_energy.items():
    commodities_df_energy[key] = yf.Ticker(value).history(period="5y")["Close"]
    
#turn commodities_dict_metals into a dataframe with closing prices
commodities_df_metals = pd.DataFrame()

for key, value in commodities_dict_metals.items():
    commodities_df_metals[key] = yf.Ticker(value).history(period="5y")["Close"]
    
#turn commodities_dict_precious_metals into a dataframe with closing prices
commodities_df_precious_metals = pd.DataFrame()

for key, value in commodities_dict_precious_metals.items():
    commodities_df_precious_metals[key] = yf.Ticker(value).history(period="5y")["Close"]
    
#turn commodities_dict_agriculture into a dataframe with closing prices
commodities_df_agriculture = pd.DataFrame()

for key, value in commodities_dict_agriculture.items():
    commodities_df_agriculture[key] = yf.Ticker(value).history(period="5y")["Close"]
    


time_frame_week = 7
time_frame_short = 21
time_frame_mid   = 50
time_frame_long = 200


In [9]:
# Combine all commodities into a single DataFrame
commodities_df_all = pd.concat([
    commodities_df_energy,
    commodities_df_metals,
    commodities_df_precious_metals,
    commodities_df_agriculture
], axis=1)

commodities_corr_energy = commodities_df_energy.corr()
commodities_corr_metals = commodities_df_metals.corr()
commodities_corr_precious_metals = commodities_df_precious_metals.corr()
commodities_corr_agriculture = commodities_df_agriculture.corr()

display(commodities_corr_energy.head())
display(commodities_corr_metals.head())
display(commodities_corr_precious_metals.head())
display(commodities_corr_agriculture.head())


# Compute correlation
commodities_corr_all = commodities_df_all.corr()

#calculate rolling 200 day correlation of all commodities
commodities_corr_all_200 = commodities_df_all.rolling(window=200).corr()

abort()
# Create a subplot with an extra space to the right for the aggregated matrix
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=("Energy", "Metals", "Precious Metals", "Agriculture", "All Commodities"),
    specs=[[{}, {}, {"rowspan":2}],
           [{}, {}, None]]
)

# Add the four individual heatmaps
fig.add_trace(go.Heatmap(
    z=commodities_corr_energy,
    x=commodities_corr_energy.columns,
    y=commodities_corr_energy.columns
), row=1, col=1)

fig.add_trace(go.Heatmap(
    z=commodities_corr_metals,
    x=commodities_corr_metals.columns,
    y=commodities_corr_metals.columns
), row=1, col=2)

fig.add_trace(go.Heatmap(
    z=commodities_corr_precious_metals,
    x=commodities_corr_precious_metals.columns,
    y=commodities_corr_precious_metals.columns
), row=2, col=1)

fig.add_trace(go.Heatmap(
    z=commodities_corr_agriculture,
    x=commodities_corr_agriculture.columns,
    y=commodities_corr_agriculture.columns
), row=2, col=2)

# Add the aggregated correlation matrix
fig.add_trace(go.Heatmap(
    z=commodities_corr_all,
    x=commodities_corr_all.columns,
    y=commodities_corr_all.columns
), row=1, col=3)

fig.update_layout(
    height=1200,
    title_text="Correlation Matrix for Commodities (with Aggregated View on the Right)"
)
fig.show()

display(commodities_corr_all.head())
#go through the commodities_corr_all dataframe and find the pairs with the highest correlation
commodities_corr_all = commodities_corr_all.stack().reset_index()
commodities_corr_all.columns = ["Commodity 1", "Commodity 2", "Correlation"]    
commodities_corr_all = commodities_corr_all[commodities_corr_all["Commodity 1"] != commodities_corr_all["Commodity 2"]]
commodities_corr_all = commodities_corr_all.sort_values("Correlation", ascending=False)

#remove duplicates, some pairs are repeated but are in different columns
commodities_corr_all = commodities_corr_all.drop_duplicates(subset=["Correlation"], keep="first")
commodities_corr_all = commodities_corr_all.reset_index(drop=True)

display(commodities_corr_all.head(20))




,Crude Oil,Natural Gas,Heating Oil,Gasoline
Crude Oil,1.000000,0.596219,0.928089,0.952809
Natural Gas,0.596219,1.000000,0.662946,0.567101
Heating Oil,0.928089,0.662946,1.000000,0.906200
Gasoline,0.952809,0.567101,0.906200,1.000000


,Gold,Silver,Copper,Platinum,Palladium
Gold,1.000000,0.822566,0.426194,-0.034548,-0.684037
Silver,0.822566,1.000000,0.611431,0.427002,-0.339535
Copper,0.426194,0.611431,1.000000,0.536413,-0.154130
Platinum,-0.034548,0.427002,0.536413,1.000000,0.396807
Palladium,-0.684037,-0.339535,-0.154130,0.396807,1.000000


,Gold,Silver,Platinum,Palladium
Gold,1.000000,0.822566,-0.034548,-0.684037
Silver,0.822566,1.000000,0.427002,-0.339535
Platinum,-0.034548,0.427002,1.000000,0.396807
Palladium,-0.684037,-0.339535,0.396807,1.000000


,Wheat,Corn,Soybeans,Soybean Oil,Soybean Meal,Oats,Rough Rice,Cotton,Oranges,Sugar,Live Cattle,Lean Hogs,Feeder Cattle
Wheat,1.000000,0.845948,0.773472,0.812720,0.601067,0.749124,0.194217,0.849132,-0.374694,0.067551,-0.194697,0.530147,-0.340280
Corn,0.845948,1.000000,0.914419,0.891314,0.741546,0.581800,0.224467,0.735955,-0.324550,0.254525,-0.053358,0.653841,-0.233044
Soybeans,0.773472,0.914419,1.000000,0.892982,0.883212,0.530941,0.276986,0.734066,-0.307433,0.343289,-0.052340,0.665698,-0.214994
Soybean Oil,0.812720,0.891314,0.892982,1.000000,0.673687,0.637077,0.262089,0.780411,-0.191225,0.391858,0.070255,0.720243,-0.076614
Soybean Meal,0.601067,0.741546,0.883212,0.673687,1.000000,0.314175,0.332716,0.516474,-0.245748,0.325918,-0.034851,0.406127,-0.201615


NameError: name 'abort' is not defined

In [ ]:
from plotly.subplots import make_subplots

#Plot Broad Commodity Prices
#Step 1: aggregate the data into a single dataframe by appending the data

commodities_df = pd.DataFrame()
commodities_sharpe_df = pd.DataFrame()
commodities_sharpe_diff_df = pd.DataFrame()

commodities_df["Broad"] = commodities_benchmark_broad["Close"]
commodities_df["Energy"] = commodities_benchmark_energy["Close"]
commodities_df["Metals"] = commodities_benchmark_metals["Close"]
commodities_df["Precious Metals"] = commodities_benchmark_precious_metals["Close"]
commodities_df["Agriculture"] = commodities_benchmark_agriculture["Close"]

def sharp_ratio(returns, rf):
    return (returns.mean() - rf) / returns.std()

market_sharpes = market_benchmark["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_sharpe_df[f"Broad 200D Sharpe"] = commodities_df["Broad"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_sharpe_df[f"Energy 200D Sharpe"] = commodities_df["Energy"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_sharpe_df[f"Metals 200D Sharpe"] = commodities_df["Metals"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_sharpe_df[f"Precious Metals 200D Sharpe"] = commodities_df["Precious Metals"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_sharpe_df[f"Agriculture 200D Sharpe"] = commodities_df["Agriculture"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))

#calculate spy minus commodity
commodities_sharpe_diff_df["Broad - SPY"] = commodities_sharpe_df["Broad 200D Sharpe"] - market_sharpes
commodities_sharpe_diff_df["Energy - Commodity"] = commodities_sharpe_df["Energy 200D Sharpe"] - commodities_sharpe_df["Broad 200D Sharpe"]
commodities_sharpe_diff_df["Metals - Commodity"] = commodities_sharpe_df["Metals 200D Sharpe"] - commodities_sharpe_df["Broad 200D Sharpe"]
commodities_sharpe_diff_df["Precious Metals - Commodity"] = commodities_sharpe_df["Precious Metals 200D Sharpe"] - commodities_sharpe_df["Broad 200D Sharpe"]
commodities_sharpe_diff_df["Precious Metals - Commodity"] = commodities_sharpe_df["Precious Metals 200D Sharpe"] - commodities_sharpe_df["Broad 200D Sharpe"]
commodities_sharpe_diff_df["Agriculture - Commodity"] = commodities_sharpe_df["Agriculture 200D Sharpe"] - commodities_sharpe_df["Broad 200D Sharpe"]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

# First subplot: 200D Sharpe Ratios
fig.add_trace(go.Scatter(x=commodities_sharpe_df.index, y=commodities_sharpe_df["Broad 200D Sharpe"], name="Broad 200D Sharpe"), row=1, col=1)
fig.add_trace(go.Scatter(x=market_sharpes.index, y=market_sharpes, name="SPY 200D Sharpe"), row=1, col=1)
fig.add_trace(go.Scatter(x=commodities_sharpe_df.index, y=[0]*len(commodities_sharpe_df), name="Zero", line=dict(color="black", width=2, dash="dash")), row=1, col=1)

# Second subplot: Broad minus SPY
fig.add_trace(go.Scatter(x=commodities_sharpe_diff_df.index, y=commodities_sharpe_diff_df["Broad - SPY"], name="Broad - SPY"), row=2, col=1)
fig.add_trace(go.Scatter(x=commodities_sharpe_diff_df.index, y=[0]*len(commodities_sharpe_diff_df), name="Zero", line=dict(color="black", width=2, dash="dash")), row=2, col=1)

fig.update_layout(
    height=1200,
    title="Comparison of 200D Sharpe Ratios",
    xaxis_title="Date (Sharpe Ratios)",
    yaxis_title="Sharpe Ratio",
    xaxis2_title="Date (Broad - SPY)",
    yaxis2_title="Broad minus SPY Sharpe Ratio"
)
fig.show()


#plot the difference in sharpe ratios
fig = go.Figure()
fig.add_trace(go.Scatter(x=commodities_sharpe_diff_df.index, y=commodities_sharpe_diff_df["Energy - Commodity"], name="Energy minus Commodity"))
fig.add_trace(go.Scatter(x=commodities_sharpe_diff_df.index, y=commodities_sharpe_diff_df["Metals - Commodity"], name="Metals minus Commodity"))
fig.add_trace(go.Scatter(x=commodities_sharpe_diff_df.index, y=commodities_sharpe_diff_df["Precious Metals - Commodity"], name="Precious Metals minus Commodity"))
fig.add_trace(go.Scatter(x=commodities_sharpe_diff_df.index, y=commodities_sharpe_diff_df["Agriculture - Commodity"], name="Agriculture minus Commodity"))
#add zero line
fig.add_trace(go.Scatter(x=commodities_sharpe_diff_df.index, y=[0]*len(commodities_sharpe_diff_df), name="Zero", line=dict(color="black", width=2, dash="dash")))
fig.update_layout(
    title="Difference between Commodity and Broad Sharpe Ratios",
    height=800
)
fig.show()



In [ ]:
#Plot Individual Commodity Prices of each commodity



In [ ]:
#Energy Commodities Fundamentals


#calculate the 200 day sharpe ratio for each commodity
commodities_energy_sharpe_df = commodities_df_energy.pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))

#plot the energy commodities sharpes
fig = go.Figure()
for column in commodities_energy_sharpe_df.columns:
    fig.add_trace(go.Scatter(x=commodities_energy_sharpe_df.index, y=commodities_energy_sharpe_df[column], name=column))
fig.add_trace(go.Scatter(x=commodities_energy_sharpe_df.index, y=[0]*len(commodities_energy_sharpe_df), name="Zero", line=dict(color="black", width=2, dash="dash")))
fig.update_layout(
    title="Energy Commodity Sharpe Ratios",
    height=800
)
fig.show()

#plot the difference in sharpe ratios between energy commodities and the broad energy index
commodities_energy_sharpe_diff_df = pd.DataFrame()
commodities_energy_sharpe_diff_df["Crude Oil minus Energy Commodity Index"] = commodities_energy_sharpe_df["Crude Oil"] - commodities_benchmark_energy["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_energy_sharpe_diff_df["Natural Gas minus Energy Commodity Index"] = commodities_energy_sharpe_df["Natural Gas"] - commodities_benchmark_energy["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_energy_sharpe_diff_df["Heating Oil minus Energy Commodity Index"] = commodities_energy_sharpe_df["Heating Oil"] - commodities_benchmark_energy["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_energy_sharpe_diff_df["Gasoline minus Energy Commodity Index"] = commodities_energy_sharpe_df["Gasoline"] - commodities_benchmark_energy["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))

fig = go.Figure()
for column in commodities_energy_sharpe_diff_df.columns:
    fig.add_trace(go.Scatter(x=commodities_energy_sharpe_diff_df.index, y=commodities_energy_sharpe_diff_df[column], name=column))
fig.add_trace(go.Scatter(x=commodities_energy_sharpe_diff_df.index, y=[0]*len(commodities_energy_sharpe_diff_df), name="Zero", line=dict(color="black", width=2, dash="dash")))
fig.update_layout(
    title="Difference between Energy Commodity and Energy Index Sharpe Ratios",
    height=800
)
fig.show()




In [ ]:
#Metals Commodities Fundamentals
commodities_metals_sharpe_df = commodities_df_metals.pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))


#plot the metals commodities sharpes
fig = go.Figure()
for column in commodities_metals_sharpe_df.columns:
    fig.add_trace(go.Scatter(x=commodities_metals_sharpe_df.index, y=commodities_metals_sharpe_df[column], name=column))
fig.add_trace(go.Scatter(x=commodities_metals_sharpe_df.index, y=[0]*len(commodities_metals_sharpe_df), name="Zero", line=dict(color="black", width=2, dash="dash")))
fig.update_layout(
    title="Metals Commodity Sharpe Ratios",
    height=800
)
fig.show()

#plot the difference in sharpe ratios between metals commodities and the broad metals index
commodities_metals_sharpe_diff_df = pd.DataFrame()
commodities_metals_sharpe_diff_df["Gold minus Metals Commodity Index"] = commodities_metals_sharpe_df["Gold"] - commodities_benchmark_metals["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_metals_sharpe_diff_df["Silver minus Metals Commodity Index"] = commodities_metals_sharpe_df["Silver"] - commodities_benchmark_metals["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_metals_sharpe_diff_df["Copper minus Metals Commodity Index"] = commodities_metals_sharpe_df["Copper"] - commodities_benchmark_metals["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_metals_sharpe_diff_df["Platinum minus Metals Commodity Index"] = commodities_metals_sharpe_df["Platinum"] - commodities_benchmark_metals["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_metals_sharpe_diff_df["Palladium minus Metals Commodity Index"] = commodities_metals_sharpe_df["Palladium"] - commodities_benchmark_metals["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))

fig = go.Figure()
for column in commodities_metals_sharpe_diff_df.columns:
    fig.add_trace(go.Scatter(x=commodities_metals_sharpe_diff_df.index, y=commodities_metals_sharpe_diff_df[column], name=column))
fig.add_trace(go.Scatter(x=commodities_metals_sharpe_diff_df.index, y=[0]*len(commodities_metals_sharpe_diff_df), name="Zero", line=dict(color="black", width=2, dash="dash")))
fig.update_layout(
    title="Difference between Metals Commodity and Metals Index Sharpe Ratios",
    height=800
)
fig.show()


In [ ]:
#Precious Metals Commodities Fundamentals

commodities_precious_metals_sharpe_df = commodities_df_precious_metals.pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))

fig = go.Figure()
for column in commodities_precious_metals_sharpe_df.columns:
    fig.add_trace(go.Scatter(x=commodities_precious_metals_sharpe_df.index, y=commodities_precious_metals_sharpe_df[column], name=column))
fig.add_trace(go.Scatter(x=commodities_precious_metals_sharpe_df.index, y=[0]*len(commodities_precious_metals_sharpe_df), name="Zero", line=dict(color="black", width=2, dash="dash")))
fig.update_layout(
    title="Precious Metals Commodity Sharpe Ratios",
    height=800
)
fig.show()

#plot the difference in sharpe ratios between precious metals commodities and the broad precious metals index
commodities_precious_metals_sharpe_diff_df = pd.DataFrame()
commodities_precious_metals_sharpe_diff_df["Gold minus Precious Metals Commodity Index"] = commodities_precious_metals_sharpe_df["Gold"] - commodities_benchmark_precious_metals["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_precious_metals_sharpe_diff_df["Silver minus Precious Metals Commodity Index"] = commodities_precious_metals_sharpe_df["Silver"] - commodities_benchmark_precious_metals["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_precious_metals_sharpe_diff_df["Platinum minus Precious Metals Commodity Index"] = commodities_precious_metals_sharpe_df["Platinum"] - commodities_benchmark_precious_metals["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_precious_metals_sharpe_diff_df["Palladium minus Precious Metals Commodity Index"] = commodities_precious_metals_sharpe_df["Palladium"] - commodities_benchmark_precious_metals["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))

fig = go.Figure()
for column in commodities_precious_metals_sharpe_diff_df.columns:
    fig.add_trace(go.Scatter(x=commodities_precious_metals_sharpe_diff_df.index, y=commodities_precious_metals_sharpe_diff_df[column], name=column))
fig.add_trace(go.Scatter(x=commodities_precious_metals_sharpe_diff_df.index, y=[0]*len(commodities_precious_metals_sharpe_diff_df), name="Zero", line=dict(color="black", width=2, dash="dash")))
fig.update_layout(
    title="Difference between Precious Metals Commodity and Precious Metals Index Sharpe Ratios",
    height=800
)
fig.show()


In [ ]:
#Agriculture Commodities Fundamentals

commodities_agriculture_sharpe_df = commodities_df_agriculture.pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))

fig = go.Figure()
for column in commodities_agriculture_sharpe_df.columns:
    fig.add_trace(go.Scatter(x=commodities_agriculture_sharpe_df.index, y=commodities_agriculture_sharpe_df[column], name=column))
    
fig.add_trace(go.Scatter(x=commodities_agriculture_sharpe_df.index, y=[0]*len(commodities_agriculture_sharpe_df), name="Zero", line=dict(color="black", width=2, dash="dash")))
fig.update_layout(
    title="Agriculture Commodity Sharpe Ratios",
    height=800
)
fig.show()

#plot the difference in sharpe ratios between agriculture commodities and the broad agriculture index
commodities_agriculture_sharpe_diff_df = pd.DataFrame()
commodities_agriculture_sharpe_diff_df["Wheat minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Wheat"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Corn minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Corn"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Soybeans minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Soybeans"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Soybean Oil minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Soybean Oil"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Soybean Meal minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Soybean Meal"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Oats minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Oats"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Rough Rice minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Rough Rice"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Cotton minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Cotton"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Oranges minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Oranges"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Sugar minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Sugar"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Live Cattle minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Live Cattle"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Lean Hogs minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Lean Hogs"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))
commodities_agriculture_sharpe_diff_df["Feeder Cattle minus Agriculture Commodity Index"] = commodities_agriculture_sharpe_df["Feeder Cattle"] - commodities_benchmark_agriculture["Close"].pct_change().rolling(200).apply(lambda x: sharp_ratio(x, 0))

fig = go.Figure()
for column in commodities_agriculture_sharpe_diff_df.columns:
    fig.add_trace(go.Scatter(x=commodities_agriculture_sharpe_diff_df.index, y=commodities_agriculture_sharpe_diff_df[column], name=column))
fig.add_trace(go.Scatter(x=commodities_agriculture_sharpe_diff_df.index, y=[0]*len(commodities_agriculture_sharpe_diff_df), name="Zero", line=dict(color="black", width=2, dash="dash")))
fig.update_layout(
    title="Difference between Agriculture Commodity and Agriculture Index Sharpe Ratios",
    height=800
)
fig.show()

